Importação das Bibliotecas

In [258]:
import pysolr
import json
import numpy as np
import logging
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Carregar variáveis globais

In [259]:
%store -r id_consulta

Configuração do Logger

In [260]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Download de dependências do NLTK e definição de stopwords

In [261]:
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    logging.info("Baixando dependências do NLTK (punkt, stopwords)...")
    nltk.download('punkt')
    nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

Carregando .jsons

In [262]:
with open('map_documentos.json', 'r', encoding='utf-8') as f:
    map_documentos = json.load(f)
with open('map_consultas.json', 'r', encoding='utf-8') as f:
    map_consultas = json.load(f)

Configuração da Conexão com o Solr

In [263]:
print("Conectando ao Solr...")
SOLR_URL_BASE = 'http://localhost:8983/solr/'
SOLR_CORE_NAME = 'documentos_ri'
SOLR_CONNECTION_STRING = f'{SOLR_URL_BASE}{SOLR_CORE_NAME}'

try:
    solr = pysolr.Solr(SOLR_CONNECTION_STRING, always_commit=True)
    solr.ping()
    logging.info(f"Conexão com o core '{SOLR_CORE_NAME}' bem-sucedida!")
except Exception as e:
    logging.error("Não foi possível conectar ao Solr. Verifique se o serviço está ativo.")
    logging.error(f"Detalhe: {e}")
    exit()


2025-07-22 11:44:19 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/admin/ping/?' (get) with body '' in 0.011 seconds, with status 200
2025-07-22 11:44:19 - INFO - Conexão com o core 'documentos_ri' bem-sucedida!


Conectando ao Solr...


Função que retorna Top N documentos de uma consulta no Solr

In [264]:
def obter_top_n_ids_solr(solr_client, consulta_tokenizada, n):
    # 1. Reconstrói a string de busca a partir dos tokens
    texto_consulta = " ".join(consulta_tokenizada)
    query_string = f'texto_documento:({texto_consulta})'

    # 2. Executa a busca no Solr, que já retorna os resultados ordenados
    resultados_solr = solr_client.search(q=query_string, fl='id', rows=n)

    # 3. Retorna apenas os 'n' primeiros IDs da lista ordenada
    # Converte o ID (string) para inteiro
    return [int(resultado['id']) for resultado in resultados_solr]


Executar consulta e gerar os resultados top 50

In [265]:
logging.info("Executando consulta de exemplo...")
# Escolha uma consulta para testar
consulta_texto = map_consultas[id_consulta]
tokens = word_tokenize(consulta_texto.lower())

consulta_tokenizada = [word for word in tokens if word.isalpha() and word not in stop_words]

logging.info(f"Consulta (ID: {id_consulta}): '{consulta_texto}'")
logging.info(f"Consulta Processada: '{' '.join(consulta_tokenizada)}'")

resultados_solr = obter_top_n_ids_solr(solr, consulta_tokenizada, 200)
logging.info(f"Encontrados {len(resultados_solr)} resultados para a consulta {id_consulta}.")



2025-07-22 11:44:19 - INFO - Executando consulta de exemplo...
2025-07-22 11:44:19 - INFO - Consulta (ID: 3): 'role apc adenomatous polyposis coli colon cancer'
2025-07-22 11:44:19 - INFO - Consulta Processada: 'role apc adenomatous polyposis coli colon cancer'
2025-07-22 11:44:19 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/select/?q=texto_documento%3A%28role+apc+adenomatous+polyposis+coli+colon+cancer%29&fl=id&rows=200&wt=json' (get) with body '' in 0.031 seconds, with status 200
2025-07-22 11:44:19 - INFO - Encontrados 200 resultados para a consulta 3.


Salvando resultados_solr como .json

In [266]:
try:
    with open('resultados_solr.json', 'w', encoding='utf-8') as f:
        json.dump(
            resultados_solr,
            f,
            indent=2,
            ensure_ascii=False
        )
    logging.info("Arquivo de resultados do Solr salvo com sucesso!")
except Exception as e:
    logging.error(f"ERRO ao salvar o arquivo JSON: {e}")
    

2025-07-22 11:44:19 - INFO - Arquivo de resultados do Solr salvo com sucesso!
